In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [2]:
### Tools' setting ###
# PATHS
owl_version = '18.06'
owl_bin     = 'owl-'+owl_version+'/bin/'
make_tgba   = 'ltl2tgba --deterministic -f %f' 
# Tools
owl         = owl_bin+'ltl2ldgba -i %f -n'
nba2ldba    = make_tgba + ' --ba | '+owl_bin+'nba2ldba'
seminator   = make_tgba + ' | ./seminator'
sem_enter   = make_tgba + ' | ./seminator --cut-on-SCC-entry'
sem_always  = make_tgba + ' | ./seminator --cut-always'
sem_bscc    = make_tgba + ' | ./seminator --bscc-avoid'
sem_bscc_en = make_tgba + ' | ./seminator --bscc-avoid --cut-on-SCC-entry'
cy          = make_tgba + ' | ./seminator --via-tba'
# Options
cut         = ' --cd'
simp_sd     = ' | autfilt --small --tgba'
simp_cd     = ' | autfilt -x simul=1,ba-simul=1 --small --tgba'
simpl_in    = ' --simplify-input'
nos         = ' -s0' # disables Spot's simplifications used in Seminator
end         = ' > %O' # saves result to file

### Ltlcross runner configuration ###
tools = {## Cut-deterministic
         'cd.ltl2ldba.no'     : owl + end,
         'cd.ltl2ldba.yes'    : owl + simp_cd + end,
         'cd.seminator.no'    : seminator + nos + cut + end,
         'cd.seminator.yes'   : seminator + cut + end,
         'cd.sem_enter.no'    : sem_enter + nos + cut + end,
         'cd.sem_enter.yes'   : sem_enter + cut + end,
         'cd.sem_always.no'   : sem_always + nos + cut + end,
         'cd.sem_always.yes'  : sem_always + cut + end,
         'cd.sem_bscc.no'     : sem_bscc + cut + nos + end,
         'cd.sem_bscc.yes'    : sem_bscc + cut + end,
         'cd.sem_bscc+en.no'  : sem_bscc_en + cut + nos + end,
         'cd.sem_bscc+en.yes' : sem_bscc_en + cut + end,
         'cd.sem_bscc+en+i.no' : sem_bscc_en + cut + simpl_in + nos + end,
         'cd.sem_bscc+en+i.yes': sem_bscc_en + cut + simpl_in + end,
         'cd.2-step.no'       : cy + nos + cut + end,     
         'cd.2-step.yes'      : cy + cut + end,
         ### Semi-deterministic
         'sd.ltl2ldba.no'     : owl + ' -n' + end,
         'sd.ltl2ldba.yes'    : owl + ' -n' + simp_sd + end,
         'sd.nba2ldba.no'     : nba2ldba + end,
         'sd.nba2ldba.yes'    : nba2ldba + simp_sd + end,
         'sd.seminator.no'    : seminator + nos + end,
         'sd.seminator.yes'   : seminator + end,
         'sd.sem_enter.no'    : sem_enter + nos + end,
         'sd.sem_enter.yes'   : sem_enter + end,
         'sd.sem_always.no'   : sem_always + nos + end,
         'sd.sem_always.yes'  : sem_always + end,
         'sd.sem_bscc.no'     : sem_bscc + nos + end,
         'sd.sem_bscc.yes'    : sem_bscc + end,
         'sd.sem_bscc+en.no'  : sem_bscc_en + nos + end,
         'sd.sem_bscc+en.yes' : sem_bscc_en + end,
         'sd.sem_bscc+en+i.no' : sem_bscc_en + simpl_in + nos + end,
         'sd.sem_bscc+en+i.yes': sem_bscc_en + simpl_in + end,
         'sd.2-step.no'       : cy + nos + end,  
         'sd.2-step.yes'      : cy + end,
        }
sd_tools = ['sd.seminator.no','sd.seminator.yes',
            'sd.sem_enter.no','sd.sem_enter.yes',
            'sd.sem_always.no','sd.sem_always.yes',
            'sd.sem_bscc.no','sd.sem_bscc.yes',
            'sd.sem_bscc+en.no','sd.sem_bscc+en.yes',
            'sd.sem_bscc+en+i.no','sd.sem_bscc+en+i.yes',
            'sd.2-step.no','sd.2-step.yes',
            'sd.ltl2ldba.no','sd.ltl2ldba.yes',
            'sd.nba2ldba.no','sd.nba2ldba.yes'
           ]
cd_tools = ['cd.seminator.no','cd.seminator.yes',
            'cd.sem_enter.no','cd.sem_enter.yes',
            'cd.sem_always.no','cd.sem_always.yes',
            'cd.sem_bscc.no','cd.sem_bscc.yes',
            'cd.sem_bscc+en.no','cd.sem_bscc+en.yes',
            'cd.sem_bscc+en+i.no','cd.sem_bscc+en+i.yes',
            'cd.2-step.no','cd.2-step.yes',
            'cd.ltl2ldba.no','cd.ltl2ldba.yes'
           ]
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = ['seminator','sem_enter','sem_always','sem_bscc','sem_bscc+en','sem_bscc+en+i','2-step','ltl2ldba','nba2ldba']

In [12]:
rerun = True

In [6]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [9]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [11]:
data = pd.DataFrame()
for (name,r) in runners.items():
    # Divide into sd_tools & cd_tools to treat timeouts separately.
    tmp_cd = pd.DataFrame(r.cummulative(tools=cd_tools),columns=[name])
    tmp_sd = pd.DataFrame(r.cummulative(tools=sd_tools),columns=[name])
    data = data.append(tmp_cd.append(tmp_sd).transpose())
data

KeyError: "['cd.sem_bscc+en+i.no' 'cd.sem_bscc+en+i.yes'] not in index"

Get the number of formulas such that all tools finished within timeout for each category.

In [ ]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

Converts the DataFrame to use MultiIndices for rows and columns

In [ ]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [ ]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [ ]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

In [ ]:
cd = tmp.cd[['']+tool_order]
sd = tmp.sd[['']+tool_order]

In [ ]:
cd

In [ ]:
sd

## Timeouts

In [17]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

Empty DataFrame
Columns: [(cd, 2-step, no), (cd, 2-step, yes), (cd, ltl2ldba, no), (cd, ltl2ldba, yes), (cd, sem_always, no), (cd, sem_always, yes), (cd, sem_bscc+en, no), (cd, sem_bscc+en, yes), (cd, sem_bscc, no), (cd, sem_bscc, yes), (cd, sem_enter, no), (cd, sem_enter, yes), (cd, seminator, no), (cd, seminator, yes), (sd, 2-step, no), (sd, 2-step, yes), (sd, ltl2ldba, no), (sd, ltl2ldba, yes), (sd, nba2ldba, no), (sd, nba2ldba, yes), (sd, sem_always, no), (sd, sem_always, yes), (sd, sem_bscc+en, no), (sd, sem_bscc+en, yes), (sd, sem_bscc, no), (sd, sem_bscc, yes), (sd, sem_enter, no), (sd, sem_enter, yes), (sd, seminator, no), (sd, seminator, yes)]
Index: []

[0 rows x 30 columns]

There is no timeout.

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [18]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.


In [19]:
display(cd)

seminator       sem_enter      sem_always      sem_bscc  \
                   n        no   yes        no  yes         no  yes       no   
origin     type                                                                
random     det   100       415   415       415  415        415  415      415   
           cd    100       464   464       464  464        464  464      464   
           sd    100       734   734       734  734        734  734      734   
           nd    100      1952  1082      1967  956       1993  977     1349   
literature det   149       556   556       556  556        556  556      556   
           cd     46       194   194       194  194        194  194      194   
           sd      3        13    13        13   13         13   13       13   
           nd     23       619   395       619  383        630  397      520   

                     sem_bscc+en      2-step       ltl2ldba       
                 yes          no  yes     no   yes       no  yes  
origin     type                                                   
random     det   415         415  415    415   415      514  460  
           cd    464         464  464    464   464      688  588  
           sd    734         734  734    734   734      921  772  
           nd    908        1364  859   2016  1111     1088  833  
literature det   556         556  556    556   556     1971  773  
           cd    194         194  194    194   194      517  331  
           sd     13          13   13     13    13       26   19  
           nd    377         520  377    625   432      453  354

In [20]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.


In [21]:
display(sd)

seminator      sem_enter      sem_always      sem_bscc  \
                   n        no  yes        no  yes         no  yes       no   
origin     type                                                               
random     det   100       415  415       415  415        415  415      415   
           cd    100       464  464       464  464        464  464      464   
           sd    100       704  704       704  704        704  704      704   
           nd    100      1145  897      1159  742       1188  735     1026   
literature det   149       556  556       556  556        556  556      556   
           cd     46       194  194       194  194        194  194      194   
           sd      3        13   13        13   13         13   13       13   
           nd     23       441  351       441  319        452  317      426   

                     sem_bscc+en      2-step      ltl2ldba      nba2ldba        
                 yes          no  yes     no  yes       no  yes       no   yes  
origin     type                                                                 
random     det   415         415  415    415  415      514  428      428   428  
           cd    464         464  464    464  464      688  545      506   506  
           sd    704         704  704    704  704      921  688      720   720  
           nd    800        1040  741   1174  927     1088  750     1643  1003  
literature det   556         556  556    556  556     1971  713      600   600  
           cd    194         194  194    194  194      517  285      207   207  
           sd     13          13   13     13   13       28   16       13    13  
           nd    336         426  319    449  386      455  327      644   411

In [25]:
runners['random_nd'].smaller_than('cd.ltl2ldba.yes','cd.sem_bscc+en.yes')

,tool,cd.ltl2ldba.yes,cd.sem_bscc+en.yes
form_id,formula,,
1,G(Fa U X(b & Fc)),6,34
2,FG(Ga U X(Ga R b)),5,7
4,G((((a & b) | (!a & !b)) & (GF!b U !c)) | (((!a & b) | (a & !b)) & (FGb R c))),9,12
7,FG(Fa U (Fb R c)),4,33
10,G(a U Xb),2,3
17,GFa | G(Gb | Xa),4,5
19,FG((F((a & G!b) | (!a & Fb)) & (!b | F(b R c))) | (b & G((!b U !c) & ((!a & G!b) | (a & Fb))))),6,12
20,G(a | X(GFa | (a M b))) R !b,3,10
32,GF((a U Xb) & G(!b | XXc)),8,12


In [28]:
runners['random_nd'].smaller_than('cd.sem_always.yes','cd.sem_bscc+en.yes')

,tool,cd.sem_always.yes,cd.sem_bscc+en.yes
form_id,formula,,
2,FG(Ga U X(Ga R b)),6,7
22,X(a & (X((G!b & XGc) | (Fb & XF!c)) R Fc)),13,14
50,(a R (Fb & Xc)) R (Gc R d),16,18
51,GF(GFa | (b & XXc)),7,9
62,((a | Fb) R c) R F(Fd R !d),20,21


In [27]:
runners['random_nd'].smaller_than('cd.sem_bscc+en.yes','cd.sem_enter.yes')

,tool,cd.sem_bscc+en.yes,cd.sem_enter.yes
form_id,formula,,
3,XFa U G(Gc | Fb),5,6
4,G((((a & b) | (!a & !b)) & (GF!b U !c)) | (((!a & b) | (a & !b)) & (FGb R c))),12,13
8,GF(a & Xb) | (c & Ga),4,7
11,XXG(Fa | Gb),6,7
12,(G!a | FGa) R Fb,5,6
14,X(G(XFa | Gb) R X!a),10,11
16,GFa R b,3,5
17,GFa | G(Gb | Xa),5,7
18,GF(a | Gb),3,4
